In [1]:
import numpy as np
import pandas as pd
import feather
import lightgbm as lgb
from lightgbm import LGBMModel, LGBMClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
#from sklearn.model_selection  import cross_val_score, StratifiedKFold
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
param = {
    'num_leaves':48,
    'min_data_in_leaf':1,
    #'num_iterations':1000,
    'learning_rate':0.01,
    'objective':'binary',
    #'objective':'regression_l2',
    'boosting':'gbdt',
    'num_threads':16,
    'device_type':'cpu',
    'max_depth':-1,
    #'valid_init_score':0.5
}



In [3]:
y = pd.read_csv("../train_numeric.csv", index_col=0, usecols=[0,969])

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
index_y1 = y[y['Response']==1].index

In [5]:
len(index_y1)

6879

In [6]:
index_y0 = y[y['Response']==0].index

In [7]:
len(index_y0)

1176868

In [8]:
important_indices_cat = np.array([1054, 1055])

In [9]:
dfs =[]

In [15]:
dfs.append(pd.read_csv("../from/train_cat_int_sub_miya.csv", index_col=0, usecols=np.concatenate([[0], important_indices_cat + 1])))

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
for i in range(0,len(dfs)):
    dfs[i] = dfs[i].reset_index().set_index("Id")
    dfs[i].index = dfs[i].index.astype(int)
    print(dfs[i].index.duplicated().sum())

0


In [13]:
# X結合
X = pd.concat(dfs, axis=1)

In [ ]:
# testデータ

In [14]:
dfs_test = []

In [27]:
dfs_test.append(pd.read_csv("../from/test_cat_int_sub_miya.csv", index_col=0, usecols=np.concatenate([[0], important_indices_cat + 1])))
dfs_test[0].head()

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,L3_S32_F3854,L3_S35_F3899
Id,,
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
5,NaN,NaN
8,NaN,NaN
10,NaN,NaN
12,NaN,NaN
15,NaN,NaN
17,NaN,NaN


In [28]:
for i in range(0,len(dfs_test)):
    dfs_test[i] = dfs_test[i].reset_index().set_index("Id")
    dfs_test[i].index = dfs_test[i].index.astype(int)
    print(dfs_test[i].index.duplicated().sum())

0
0


In [29]:
X_test = pd.concat(dfs_test, axis=1)
X_test.head()

,L3_S32_F3854,L3_S35_F3899,L3_S32_F3854,L3_S35_F3899
Id,,,,
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN


In [24]:
y_train = y.astype(np.float32).values.ravel()
print(y_train)

[0. 0. 0. ... 0. 0. 0.]


In [25]:
X_mean = X.mean()
X_mean.head()

L3_S32_F3854   NaN
L3_S35_F3899   NaN
dtype: float64

In [30]:
%%time
n_folds=5

cv = StratifiedKFold(y_train, n_folds)
models = []
preds_oof  = np.ones(y_train.shape[0])
preds_test = np.empty((n_folds, X_test.shape[0]))

for i, (train, valid) in enumerate(cv):
    train_data = lgb.Dataset(X.iloc[train], y_train[train])
    test_data = lgb.Dataset(X.iloc[valid], y_train[valid])

    model = lgb.train(param, train_data, 1000, early_stopping_rounds=50, valid_sets=[test_data])
    
    preds_oof[valid] = model.predict(X.iloc[valid])
    preds_test[i, :] = model.predict(X_test)

    models.append(model)

ValueError: DataFrame.dtypes for data must be int, float or bool. Did not expect the data types in fields L3_S32_F3854, L3_S35_F3899

In [ ]:
lgb.plot_importance(model, max_num_features=100, figsize=(20,20))

In [ ]:
%%time
plt.figure()
thresholds = np.linspace(0.01, 1.00, 100)
mcc = np.array([metrics.matthews_corrcoef(y, preds_oof>thr) for thr in thresholds])
plt.plot(thresholds, mcc)
best_threshold = thresholds[mcc.argmax()]
print("MCC: %f" % mcc.max())
print("best_threshold: %f" % best_threshold)

In [ ]:
preds_oof

In [ ]:
preds = (preds_test.mean(axis=0) > best_threshold).astype(np.int8)
sub = pd.read_csv("sample_submission.csv", index_col=0)
sub["Response"] = preds
sub.to_csv("submission.csv.gz", compression="gzip")